# RAG Query Pipeline
## 1. Imports thư viện cần thiết

In [1]:
# pip install langchain-groq
import os
import chromadb
from langchain_groq import ChatGroq
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("✅ Đã import thành công các thư viện cần thiết cho RAG pipeline.")

✅ Đã import thành công các thư viện cần thiết cho RAG pipeline.


## 2. Khởi tạo Mô hình (LLM và Embedding)

In [2]:
# Cấu hình biến môi trường cho Groq API Key trong file .env
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
print("✅ Đã cấu hình biến môi trường GROQ_API_KEY từ file .env.")

# 1. Khởi tạo 3 LLM chúng ta muốn so sánh
# temperature là tham số điều chỉnh độ sáng tạo của câu trả lời
llm_llama_8b = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0 # = 0 để câu trả lời bám sát context nhất có thể
)
print(f"✅ Đã khởi tạo LLM 1: llama-3.1-8b-instant")

llm_llama_70b = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)
print(f"✅ Đã khởi tạo LLM 2: llama-3.3-70b-versatile")

llm_compound = ChatGroq(
    model="groq/compound",
    temperature=0
)
print(f"✅ Đã khởi tạo LLM 3: groq/compound")

# 2. Khởi tạo Embedding Model (Dùng lại model cũ ở file 01)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Đã khởi tạo Embedding Model (multilingual-MiniLM).")

✅ Đã cấu hình biến môi trường GROQ_API_KEY từ file .env.
✅ Đã khởi tạo LLM 1: llama-3.1-8b-instant
✅ Đã khởi tạo LLM 2: llama-3.3-70b-versatile
✅ Đã khởi tạo LLM 3: groq/compound
✅ Đã khởi tạo Embedding Model (multilingual-MiniLM).


## 3. Tải Vector Store

In [3]:
# Tải lại database đã được xây dựng và LỌC SẠCH ở file 01
db_path = "./chroma_db"
collection_name = "academic_regulations" # Đảm bảo tên này khớp với file 01

vectorstore = Chroma(
    persist_directory=db_path,
    embedding_function=embedding_model,
    collection_name=collection_name
)

# Tạo một "retriever" để lấy tài liệu
# search_kwargs={"k": 5}: Lấy 5 chunk liên quan nhất
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print(f"✅ Đã tải thành công ChromaDB từ '{db_path}' (collection: '{collection_name}')")
print(f"📚 Số lượng chunks trong DB: {vectorstore._collection.count()}")

✅ Đã tải thành công ChromaDB từ './chroma_db' (collection: 'academic_regulations')
📚 Số lượng chunks trong DB: 156


## 4. Tạo Chuỗi RAG Hoàn chỉnh (End-to-End)

In [4]:
from langchain_core.runnables import RunnableParallel

# 1. Tạo prompt RAG chính
# Ra lệnh cho LLM trả lời DỰA TRÊN context được cung cấp
rag_template = """
Bạn là một trợ lý AI hữu ích, chuyên trả lời các câu hỏi về quy định học thuật của trường 
ĐH Sư Phạm Kỹ Thuật TP.HCM dựa trên các văn bản được cung cấp.
Hãy trả lời câu hỏi của người dùng một cách ngắn gọn và chính xác, 
chỉ dựa vào nội dung trong phần "Văn bản tham khảo" dưới đây.
KHÔNG được bịa đặt thông tin. Nếu không tìm thấy, hãy nói "Tôi không tìm thấy thông tin này trong tài liệu."

Văn bản tham khảo:
{context}

Câu hỏi:
{question}

Câu trả lời (chỉ dựa trên văn bản):
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

original_context=""  # Biến toàn cục để lưu context sau khi gộp
# 2. Hàm này để gộp các chunk tìm được thành 1 đoạn text
def format_context(docs):
    global original_context
    context_parts = []
    
    for i, doc in enumerate(docs):
        # Lấy tên file từ metadata (chuẩn hóa dấu \ cho đẹp)
        source_file = doc.metadata.get('source', 'Không rõ nguồn').replace('\\', '/')
        
        # Tạo một khối context có đánh số và ghi nguồn
        part = f"""
--- NGUỒN THAM KHẢO #{i+1} ---
Nguồn file: {source_file}
Nội dung:
{doc.page_content}
"""
        context_parts.append(part)
    
    # Gộp tất cả lại
    original_context = "\n".join(context_parts)
    return original_context

# # 3. TẠO CHUỖI RAG HOÀN CHỈNH khi chạy 1 model
# # Đây là "công thức" của RAG:
# # 1. Lấy câu hỏi gốc (RunnablePassthrough)
# # 2. Dùng câu hỏi gốc để tìm context (retriever)
# # 3. Gộp context lại (format_context) và lưu vào biến "context"
# # 4. Nhồi context và câu hỏi gốc vào RAG prompt (rag_prompt)
# # 5. Đưa cho LLM trả lời (llm)
# # 6. Lấy kết quả (StrOutputParser)

# rag_chain = (
#     {
#         "context": RunnablePassthrough() 
#                    | retriever 
#                    | format_context,
#         "question": RunnablePassthrough()
#     }
#     | rag_prompt
#     | llm
#     | StrOutputParser()
# )

# print("✅ Đã tạo chuỗi RAG hoàn chỉnh.")

In [5]:
# Truong
# 3. TÁCH CHUỖI:
# Chuỗi 1: Chỉ để lấy context (Chạy 1 lần)


# Chuỗi 2: Chỉ để trả lời (Dùng 3 lần với 3 LLM)
# ({context, question} -> prompt -> llm -> parser)
# answer_chain_8b = rag_prompt | llm_llama_8b | StrOutputParser()
# answer_chain_70b = rag_prompt | llm_llama_70b | StrOutputParser()
# answer_chain_compound = rag_prompt | llm_compound | StrOutputParser()

# # 4. GỘP CÁC CHUỖI TRẢ LỜI LẠI ĐỂ CHẠY SONG SONG
# parallel_answer_chains = RunnableParallel(
#     llama_8b=answer_chain_8b,
#     llama_70b=answer_chain_70b,
#     compound=answer_chain_compound
# )

# print("✅ Đã tạo chuỗi Retrieval và 3 chuỗi Answer song song.")

## 5. Chạy Thử nghiệm Toàn bộ Pipeline

In [6]:
# # Đặt câu hỏi (viết tắt)
# # user_query = "thời gian đkmh là khi nào?"
# user_query = "Mức điểm các loại chứng chỉ ngoại ngữ được sử dụng để xét miễn học các phần Kỹ năng giao tiếp tiếng Anh?"
# print(f"🔍 Đang thực hiện truy vấn RAG cho câu hỏi: '{user_query}'")
# print("\n" + "="*80)

# # Dùng .stream() để xem LLM "gõ" câu trả lời
# full_response = ""

# for chunk in rag_chain.stream(user_query):
#     print(chunk, end="", flush=True)
#     full_response += chunk

# print("\n" + "="*80)
# print("\n✅ Hoàn tất!")

In [7]:
# Đặt câu hỏi (Giữ nguyên câu hỏi lỗi để test)
user_query = "Mức điểm các loại chứng chỉ ngoại ngữ được sử dụng để xét miễn học các phần Kỹ năng giao tiếp tiếng Anh?"
# user_query = "Điều kiện về Tiếng Anh để xét tốt nghiệp cho sinh viên khóa 2020 trở về trước là gì?"
user_query = "sv hệ clc có được đk học với lớp đại trà không?" # Từ viết tắt - Câu hỏi hơi khó


# Hàm xử lý stream và in từng chunk ngay khi nhận được
def process_stream(generator, model_name):
    print(f"\n🤖 {model_name}: ", end="", flush=True)  # Hiển thị tên model
    for chunk in generator:
        if hasattr(chunk, "content"):
            print(chunk.content, end="", flush=True)  # In từng phần nội dung
    print()  # Xuống dòng sau khi hoàn tất

# 3. Chạy thử nghiệm tuần tự với stream
print(f"🔍 Đang thực hiện truy vấn RAG (stream) cho câu hỏi: '{user_query}'")
print("\n" + "="*80)

# Bước 1: Lấy context (Chỉ chạy 1 lần)
print("⏳ Bước 1: Đang lấy context từ ChromaDB...")
# (Câu hỏi -> retriever -> format_context)
retrieval_chain = retriever | format_context
context = retrieval_chain.invoke(user_query)
print("✅ Đã lấy context.")

# Bước 2: Chạy từng model tuần tự với stream
print("⏳ Bước 2: Đang chạy các model tuần tự với stream...")

# Định dạng đầu vào bằng ChatPromptTemplate
formatted_input = rag_prompt.format(context=context, question=user_query)

# Chạy từng model và stream kết quả
process_stream(llm_llama_8b.stream(formatted_input), "Llama 3.1 8B (Instant)")
process_stream(llm_llama_70b.stream(formatted_input), "Llama 3.3 70B (Versatile)")
process_stream(llm_compound.stream(formatted_input), "Groq Compound")

print("\n✅ Hoàn tất!")

🔍 Đang thực hiện truy vấn RAG (stream) cho câu hỏi: 'sv hệ clc có được đk học với lớp đại trà không?'

⏳ Bước 1: Đang lấy context từ ChromaDB...
✅ Đã lấy context.
⏳ Bước 2: Đang chạy các model tuần tự với stream...

🤖 Llama 3.1 8B (Instant): Tôi không tìm thấy thông tin này trong tài liệu.

🤖 Llama 3.3 70B (Versatile): Sinh viên hệ CLC không được đăng ký học với các lớp Đại trà hoặc ngược lại, trừ các trường hợp đặc biệt phải làm đơn và được sự đồng ý của BCN khoa quản ngành và phòng Đào tạo.

🤖 Groq Compound: **Trả lời:**  
Không, sinh viên hệ CLC không được đăng ký học với lớp đại trà.  

**Lý do (theo Văn bản tham khảo #2):**  
- “SV hệ CLC không được đăng ký học với các lớp Đại trà hoặc ngược lại. Các trường hợp đặc biệt phải làm đơn và được sự đồng ý của Ban Chấp hành khoa, quản ngành và phòng Đào tạo.”  

Vì vậy, việc đăng ký chung chỉ được thực hiện khi có giấy phép đặc biệt từ các cơ quan trên.

✅ Hoàn tất!


In [9]:
print(f"Context gửi đi ({len(original_context)} ký tự) có số lượng tokens tối đa là: {llm_compound.get_num_tokens(original_context)} tokens")
print(f"Nội dung context gửi đi:\n{original_context}")

Token indices sequence length is longer than the specified maximum sequence length for this model (4008 > 1024). Running this sequence through the model will result in indexing errors


Context gửi đi (4714 ký tự) có số lượng tokens tối đa là: 4008 tokens
Nội dung context gửi đi:

--- NGUỒN THAM KHẢO #1 ---
Nguồn file: ./data/2692_Thông báo vv kế hoạch đăng ký và chỉnh sửa học phần qua mạng học kỳ I, năm học 2025-2026 (dành cho sinh viên chính quy hệ đại trà và CLC TV tại trường).pdf
Nội dung:
-_ Phòng Đào tạo không đăng ký sẵn cho SV. SV có thể tự do lựa chọn nhóm lớp để phù hợp với TKB của mình.
-_ Từ học kỳ 1 năm học 2025-2026 sinh viên tất cả các khóa sẽ học theo mã giáo dục thể chất mới (thay thế cho các mã giáo dục thể chất cũ) như sau:
FOOT112330 Bóng đá Hai học phần Giáo dục thể BASKI12330 | Bóng rổ chất 2 và Giáo BADM112330 | Cầu lông St Si6ĐÁ| - (Sinh viên chọn | TENNI 2n lam vợt ọo 2 học phân KARA112330 | Không thủ đạo trong 08 học VOLLI12330 | Bóng chuyền *k CHESI12330 | Cờ vua phần lựa chọn theo danh sách) PICKI12330 | Pickleball
6.. Đôi với các học phân chính trị từ khóa 2019 trở về sau: SV phải đăng ký theo danh sách
các học phần và thứ tự tiên quyết sa